# Notes

If using perfect approximate georeferencing (exactgeo),
everything works perfectly,
even for large paddings (up to 2).


With the following settings we can get some pretty good results.

- Only referenced images.
- Exact georeferencing with small padding.

The following variations on training sample.

- Near perfect: 10 training samples per camera.
- Very good: 10 training samples for the nadir camera. 1 training sample per other camera.
- Good: 10 training samples for the nadir camera only.
- Okay: 5 training samples per camera.


With the following settings we can get some pretty good results.

- Only referenced images.
- Approximate georeferencing with 0.5 padding.
- 10 training samples per camera

The following variations on training sample.

- 1.0 padding


# Setup


## Imports


In [ ]:
import os
import warnings

In [ ]:
import numpy as np
import pandas as pd
import yaml

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from night_horizons import utils

## Settings


In [ ]:
with open('./config.yml', "r", encoding='UTF-8') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
local_settings = {
    'expected_count': 15000,
}
settings.update(local_settings)

In [ ]:
# Until I reinstall seaborn, this helps things be less annoying...
warnings.simplefilter(action='ignore', category=FutureWarning)

# Extract, Transform, Load


## Load Data


In [ ]:
# Discover y pred data
data = {}
y_pred_fps = utils.discover_data(settings['data_dir'], ['csv'], 'y_pred')
data['y_pred'] = [pd.read_csv(_, index_col=0) for _ in y_pred_fps]

In [ ]:
# Get the other data
for var_name in ['y_train', 'y_test', 'X_train', 'X_test', 'y_pred']:
    var_fps = y_pred_fps.str.replace('y_pred', var_name)
    data[var_name] = [pd.read_csv(_, index_col=0) for _ in var_fps]

In [ ]:
# Now get settings
mosaic_settings_fps = y_pred_fps.str.replace('y_pred.csv', 'settings.yaml')
nb_settings_fps = y_pred_fps.str.replace('y_pred.csv', 'nbsettings.yaml')

mosaic_settings = []
nb_settings = []
for i, settings_fp in enumerate(mosaic_settings_fps):
    with open(settings_fp, "r", encoding='UTF-8') as file:
        mosaic_settings_i = yaml.load(file, Loader=yaml.FullLoader)
    mosaic_settings.append(mosaic_settings_i)

    with open(nb_settings_fps.iloc[i], "r", encoding='UTF-8') as file:
        nb_settings_i = yaml.load(file, Loader=yaml.FullLoader)
    nb_settings.append(nb_settings_i)
    

In [ ]:
# Finally, the logs
log_fps = y_pred_fps.str.replace('y_pred.csv', 'log.csv')
logs = [pd.read_csv(_, index_col=0) for _ in log_fps]

## Extract Quantities


In [ ]:
df = pd.DataFrame({'y_pred_fp': y_pred_fps})

### Settings


In [ ]:
# Relevant settings
ts_cols = [f'cam{j}_train_size' for j in np.arange(3)]
for i, nb_settings_i in enumerate(nb_settings):
    for j in np.arange(3):
        df.loc[i, ts_cols[j]] = nb_settings_i['train_size'][j]

    for setting in ['padding_fraction', 'use_approximate_georeferencing']:
        df.loc[i, setting] = nb_settings_i[setting]

df['train_size'] = df[ts_cols].sum(axis=1)

### Evaluation Metrics


In [ ]:
# Return codes
for i, log in enumerate(logs):
    ret_counts = log['return_code'].value_counts().astype(int)
    cols = [_ + '_count' for _ in ret_counts.index]
    df.loc[i, cols] = ret_counts.values
    df.loc[i, 'count'] = len(log)

In [ ]:
# Fractions
for col in df.columns:
    if not '_count' in col:
        continue
    df[col.replace('count', 'frac')] = df[col] / df['count']

In [ ]:
# Offset
for i, y_pred in enumerate(data['y_test']):
    df.loc[i, 'offset_mean'] = y_pred['offset'].mean()
    df.loc[i, 'offset_low'] = np.nanpercentile(y_pred['offset'], 16.)
    df.loc[i, 'offset_high'] = np.nanpercentile(y_pred['offset'], 84.)

In [ ]:
# Durations
for i, log in enumerate(logs):
    df.loc[i, 'avg_join_duration'] = log['duration'].mean()
    df.loc[i, 'avg_iter_duration'] = log['iter_duration'].mean()
    df.loc[i, 'total_iter_duration'] = log['iter_duration'].sum()
df['expected_iter_duration_hr'] = np.round(
    df['avg_iter_duration'] * settings['expected_count'] / 3600.,
    1
)

# Summarize


In [ ]:
# Make the overall grid
g = sns.PairGrid(
    data=df,
    x_vars=['train_size', 'padding_fraction'],
    y_vars=['success_frac', 'offset_mean', 'offset_low', 'offset_high'],
    hue='use_approximate_georeferencing',
)
g.map(sns.scatterplot)
g.add_legend()

# Adjust limits
# Success fraction limits
for ax in g.axes[0]:
    ax.set_ylim(0, 1)
# Offset limits
for ax in g.axes[1]:
    ax.set_ylim(0, ax.get_ylim()[1])
for ax in g.axes[2]:
    ax.set_ylim(0, ax.get_ylim()[1])
for ax in g.axes[3]:
    ax.set_ylim(0, ax.get_ylim()[1])